In [ ]:
# QUICK API TEST - Run this first to verify your API key works
import requests
import json

# Your API key - set as environment variable for security
import os
AFDC_API_KEY = os.getenv('AFDC_API_KEY', 'DEMO_KEY')  # Use DEMO_KEY as fallback for testing

# Check if API key is properly set
if AFDC_API_KEY == 'DEMO_KEY':
    print("⚠️  WARNING: Using DEMO_KEY - set AFDC_API_KEY environment variable for full functionality")
    print("   To set: export AFDC_API_KEY='your_actual_key_here'")
else:
    print("✅ AFDC_API_KEY loaded from environment")

def test_api_key():
    """Quick test to verify API key works by fetching a small amount of data."""
    print("Testing AFDC API key...")
    
    # Test 1: Get EV networks (small, fast request)
    try:
        url = "https://developer.nrel.gov/api/alt-fuel-stations/v1/electric-networks.json"
        params = {
            "api_key": AFDC_API_KEY,
            "country": "US"
        }
        
        print("Making test request to EV networks endpoint...")
        response = requests.get(url, params=params, timeout=10)
        
        if response.status_code == 200:
            networks = response.json()
            print(f"API key works! Found {len(networks)} EV networks")
            
            # Show a few network names
            print("Sample networks:")
            for i, network in enumerate(networks[:5]):
                print(f"  - {network.get('name', 'Unknown')}")
            
        else:
            print(f"API request failed with status code: {response.status_code}")
            print(f"Response: {response.text}")
            return False
            
    except Exception as e:
        print(f"Error making API request: {e}")
        return False
    
    # Test 2: Get just 5 stations near San Diego (quick test)
    try:
        print("\nTesting station data retrieval (5 stations only)...")
        url = "https://developer.nrel.gov/api/alt-fuel-stations/v1/nearest.json"
        params = {
            "api_key": AFDC_API_KEY,
            "latitude": 32.834,
            "longitude": -117.123,
            "radius": 10,  # Small radius for quick test
            "fuel_type": "ELEC",
            "limit": 5     # Only 5 stations for quick test
        }
        
        response = requests.get(url, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            stations = data.get("fuel_stations", [])
            print(f"Found {len(stations)} test stations near San Diego")
            
            if stations:
                print("Sample station:")
                station = stations[0]
                print(f"  - Name: {station.get('station_name', 'Unknown')}")
                print(f"  - Network: {station.get('ev_network', 'Unknown')}")
                print(f"  - City: {station.get('city', 'Unknown')}")
                print(f"  - DC Fast: {station.get('ev_dc_fast_num', 0)}")
                print(f"  - Level 2: {station.get('ev_level2_evse_num', 0)}")
            
            print("\nAPI is working correctly! You can now run the full script.")
            return True
        else:
            print(f"Station request failed with status code: {response.status_code}")
            print(f"Response: {response.text}")
            return False
            
    except Exception as e:
        print(f"Error getting station data: {e}")
        return False

# Run the test
test_api_key()


Testing AFDC API key...
Making test request to EV networks endpoint...
✅ API key works! Found 82 EV networks
Sample networks:
  - All
  - 7Charge
  - ABM
  - AmpedUp! Networks
  - AmpUp

Testing station data retrieval (5 stations only)...
✅ Found 5 test stations near San Diego
Sample station:
  - Name: Viewridge MOB - CA205-1
  - Network: Blink Network
  - City: San Diego
  - DC Fast: None
  - Level 2: 3

🎉 API is working correctly! You can now run the full script.


True

In [ ]:
"""
Pulls AFDC EV station metadata for San Diego County, excludes Tesla networks,
and writes two CSVs:
  - data/afdc_stations_raw.csv
  - data/afdc_stations_top50.csv

Docs:
- Nearest Stations: https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/nearest/
- EV Charging Networks: https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/electric-networks/
"""

import os
import sys
import json
import time
import math
import requests
import pandas as pd
from dateutil import parser as dtp

# =========================
# CONFIG — EDIT ME
# =========================
AFDC_API_KEY = os.getenv('AFDC_API_KEY', 'DEMO_KEY')  # Load from environment variable
OUT_DIR = "."  # Current directory (data/afdc/)
os.makedirs(OUT_DIR, exist_ok=True)

# San Diego region center & radius (miles)
SD_LAT, SD_LON = 32.834, -117.123   # central-ish SD County
SD_RADIUS_MILES = 80                # wide enough to cover the county (adjust if needed)

# Station filters
ACCESS = "public"                   # public access only
STATUS = "E"                        # E = Available
FUEL = "ELEC"                       # Electric only

# Initial top-50 proxy score weights
W_DC = 1.0
W_L2 = 0.25

# =========================
# Helpers
# =========================
BASE = "https://developer.nrel.gov/api/alt-fuel-stations/v1"

def _check_key():
    if not AFDC_API_KEY or AFDC_API_KEY == "PUT_YOUR_NREL_KEY_HERE":
        sys.exit("ERROR: Set AFDC_API_KEY env var or edit the script to include your NREL key.")

def fetch_ev_networks(country="US", include_inactive=False):
    """Fetch all EV charging networks from AFDC API."""
    url = f"{BASE}/electric-networks.json"
    params = {
        "api_key": AFDC_API_KEY,
        "country": country,
        "include_inactive": str(include_inactive).lower()
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def get_tesla_network_keys(network_json):
    """Return a set of network 'key' values whose names include 'Tesla'."""
    tesla = set()
    for item in network_json:
        name = (item.get("name") or "").lower()
        key = item.get("key")
        if "tesla" in name and key:
            tesla.add(key)
    return tesla

def fetch_nearest_stations(lat, lon, radius_miles, fuel_type="ELEC", access="public", status="E", limit=200):
    """
    Nearest endpoint returns up to 'limit' stations. If we need more, we can
    increase radius or call multiple tiles. For SD County, 80 mi radius is ample.
    """
    url = f"{BASE}/nearest.json"
    params = {
        "api_key": AFDC_API_KEY,
        "latitude": lat,
        "longitude": lon,
        "radius": radius_miles,
        "fuel_type": fuel_type,
        "access": access,
        "limit": limit  # Remove status parameter - it might be causing issues
    }
    
    # Only add status if it's a valid value
    if status and status in ["E", "P", "T"]:  # E=Available, P=Planned, T=Temporarily unavailable
        params["status"] = status
    
    print(f"API request params: {params}")  # Debug output
    
    try:
        r = requests.get(url, params=params, timeout=45)
        print(f"Response status code: {r.status_code}")
        
        if r.status_code == 422:
            print(f"422 Error response: {r.text}")
            # Try without status parameter
            print("Retrying without status parameter...")
            params_no_status = params.copy()
            if "status" in params_no_status:
                del params_no_status["status"]
            r = requests.get(url, params=params_no_status, timeout=45)
            print(f"Retry response status code: {r.status_code}")
        
        r.raise_for_status()
        return r.json()
        
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response content: {r.text if 'r' in locals() else 'No response'}")
        raise

def normalize_record(s):
    """Flatten AFDC station record into a friendly dict."""
    # Known fields we care about (present in nearest/all responses)
    fields = {
        "id": s.get("id"),
        "station_name": s.get("station_name"),
        "ev_network": s.get("ev_network"),
        "city": s.get("city"),
        "state": s.get("state"),
        "zip": s.get("zip"),
        "street_address": s.get("street_address"),
        "latitude": s.get("latitude"),
        "longitude": s.get("longitude"),
        "ev_connector_types": ",".join(s.get("ev_connector_types") or []) if isinstance(s.get("ev_connector_types"), list) else s.get("ev_connector_types"),
        "ev_dc_fast_num": s.get("ev_dc_fast_num"),
        "ev_level1_evse_num": s.get("ev_level1_evse_num"),
        "ev_level2_evse_num": s.get("ev_level2_evse_num"),
        "ev_pricing": s.get("ev_pricing"),
        "access_days_time": s.get("access_days_time"),
        "access_code": s.get("access_code"),
        "date_last_confirmed": s.get("date_last_confirmed"),
        "updated_at": s.get("updated_at"),
        "open_date": s.get("open_date"),
        "geocode_status": s.get("geocode_status"),
    }
    return fields

def compute_capacity_proxy(row, w_dc=W_DC, w_l2=W_L2):
    """Compute a capacity proxy score based on DC fast and Level 2 chargers."""
    dc = row.get("ev_dc_fast_num") or 0
    l2 = row.get("ev_level2_evse_num") or 0
    # guard if they are strings
    try:
        dc = int(dc)
    except Exception:
        dc = 0
    try:
        l2 = int(l2)
    except Exception:
        l2 = 0
    return w_dc * dc + w_l2 * l2

In [ ]:
# =========================
# Main Function
# =========================
def main():
    """Main function to fetch and process AFDC station data for San Diego County."""
    _check_key()

    print("Fetching EV networks to identify Tesla…")
    networks = fetch_ev_networks(country="US", include_inactive=False)
    tesla_keys = get_tesla_network_keys(networks)
    print(f"Tesla-like network keys: {sorted(tesla_keys)}")

    print(f"Fetching AFDC stations near San Diego (radius={SD_RADIUS_MILES} mi)…")
    payload = fetch_nearest_stations(
        lat=SD_LAT,
        lon=SD_LON,
        radius_miles=SD_RADIUS_MILES,
        fuel_type=FUEL,
        access=ACCESS,
        status=STATUS,
        limit=200  # Reduced limit - API might have max limit of 200
    )

    stations = payload.get("fuel_stations", [])
    if not stations:
        sys.exit("No stations returned. Try increasing radius or check your API key/filters.")

    print(f"Raw stations returned: {len(stations)}")

    # Normalize and de-duplicate by AFDC station id
    rows = [normalize_record(s) for s in stations]
    df = pd.DataFrame(rows).drop_duplicates(subset=["id"]).reset_index(drop=True)

    # Exclude Tesla networks using keys or name contains "Tesla"
    # AFDC records here have 'ev_network' as human-readable name, not the key.
    # So we do a name-based exclusion as well.
    df["ev_network_lower"] = df["ev_network"].str.lower().fillna("")
    df_filt = df[~df["ev_network_lower"].str.contains("tesla")].copy()
    df_filt.drop(columns=["ev_network_lower"], inplace=True)
    print(f"After excluding Tesla networks: {len(df_filt)} stations")

    # Save raw CSV
    raw_path = os.path.join(OUT_DIR, "afdc_stations_raw.csv")
    df_filt.to_csv(raw_path, index=False)
    print(f"Saved: {raw_path}")

    # Compute initial "busiest proxy" and take top 50
    df_filt["capacity_proxy"] = df_filt.apply(compute_capacity_proxy, axis=1)
    df_top = df_filt.sort_values("capacity_proxy", ascending=False).head(50).reset_index(drop=True)

    top_path = os.path.join(OUT_DIR, "afdc_stations_top50.csv")
    df_top.to_csv(top_path, index=False)
    print(f"Saved: {top_path}")

    # Quick stats
    print("\nSummary:")
    print("Top 10 stations by capacity proxy:")
    print(df_top[["station_name", "city", "ev_network", "ev_dc_fast_num", "ev_level2_evse_num", "capacity_proxy"]].head(10))
    
    print(f"\nTotal stations found: {len(df_filt)}")
    print(f"Top 50 stations saved with capacity proxy scores")
    print(f"Networks represented: {df_filt['ev_network'].nunique()}")
    print(f"Cities covered: {df_filt['city'].nunique()}")
    
    return df_filt, df_top

# Run the main function
if __name__ == "__main__":
    df_all, df_top50 = main()


Fetching EV networks to identify Tesla…
Tesla-like network keys: ['Tesla', 'Tesla Destination']
Fetching AFDC stations near San Diego (radius=80 mi)…
API request params: {'api_key': 'X4hvpURj6jcmyUGiQmiDMGLk3LD5Nrs9W2fQu4bu', 'latitude': 32.834, 'longitude': -117.123, 'radius': 80, 'fuel_type': 'ELEC', 'access': 'public', 'limit': 200, 'status': 'E'}
Response status code: 200
Raw stations returned: 200
After excluding Tesla networks: 186 stations
✅ Saved: ./afdc_stations_raw.csv
✅ Saved: ./afdc_stations_top50.csv

Summary:
Top 10 stations by capacity proxy:
                        station_name       city         ev_network  \
0        Fashion Valley Mall – South  San Diego  Electrify America   
1               Vons San Diego #2358  San Diego       eVgo Network   
2                   Stonecrest Plaza  San Diego       eVgo Network   
3     Mission Valley (San Diego, CA)  San Diego  Electrify America   
4      Mission Valley Office Complex  San Diego         EV Connect   
5  Sheraton Miss

In [ ]:
# Test the script and show additional analysis
# Check if main function exists (make sure you ran the previous cells first)
try:
    print("Running AFDC data collection script...")
    df_all_stations, df_top_50 = main()
except NameError as e:
    print(f"Error: {e}")
    print("🔧 Please run the cells in order:")
    print("   1. Cell 0: Quick API test")
    print("   2. Cell 1: Function definitions") 
    print("   3. Cell 2: Main execution")
    print("   4. Cell 3: This analysis cell")
    print("\nMake sure to run Cell 1 (the long cell with all the function definitions) before running this cell!")
    raise

# Additional analysis
print("\n" + "="*50)
print("ADDITIONAL ANALYSIS")
print("="*50)

# Network distribution
print("\nNetwork Distribution (Top 10):")
network_counts = df_all_stations['ev_network'].value_counts().head(10)
print(network_counts)

# Geographic distribution
print("\nCity Distribution (Top 10):")
city_counts = df_all_stations['city'].value_counts().head(10)
print(city_counts)

# Connector types analysis
print("\nConnector Types Analysis:")
connector_counts = df_all_stations['ev_connector_types'].value_counts().head(10)
print(connector_counts)

# Capacity analysis
print(f"\nCapacity Analysis:")
print(f"Average DC Fast Chargers per station: {df_all_stations['ev_dc_fast_num'].fillna(0).astype(float).mean():.2f}")
print(f"Average Level 2 Chargers per station: {df_all_stations['ev_level2_evse_num'].fillna(0).astype(float).mean():.2f}")
print(f"Stations with DC Fast Charging: {(df_all_stations['ev_dc_fast_num'].fillna(0).astype(float) > 0).sum()}")
print(f"Stations with Level 2 only: {((df_all_stations['ev_level2_evse_num'].fillna(0).astype(float) > 0) & (df_all_stations['ev_dc_fast_num'].fillna(0).astype(float) == 0)).sum()}")

print(f"\nFiles created:")
print(f"- afdc_stations_raw.csv: {len(df_all_stations)} stations")
print(f"- afdc_stations_top50.csv: {len(df_top_50)} stations")


Running AFDC data collection script...
Fetching EV networks to identify Tesla…
Tesla-like network keys: ['Tesla', 'Tesla Destination']
Fetching AFDC stations near San Diego (radius=80 mi)…
API request params: {'api_key': 'X4hvpURj6jcmyUGiQmiDMGLk3LD5Nrs9W2fQu4bu', 'latitude': 32.834, 'longitude': -117.123, 'radius': 80, 'fuel_type': 'ELEC', 'access': 'public', 'limit': 200, 'status': 'E'}
Response status code: 200
Raw stations returned: 200
After excluding Tesla networks: 186 stations
✅ Saved: ./afdc_stations_raw.csv
✅ Saved: ./afdc_stations_top50.csv

Summary:
Top 10 stations by capacity proxy:
                        station_name       city         ev_network  \
0        Fashion Valley Mall – South  San Diego  Electrify America   
1               Vons San Diego #2358  San Diego       eVgo Network   
2                   Stonecrest Plaza  San Diego       eVgo Network   
3     Mission Valley (San Diego, CA)  San Diego  Electrify America   
4      Mission Valley Office Complex  San Diego